In [29]:
from pamda import pamda

In [30]:
from math import log

def bmssp_big_o(n, m):
    # BMSSP has a time complexity of O(m log(n)^0.66)
    return m * (log(n)**.66)

def dijkstra_big_o(n, m):
    # Dijkstra's algorithm as implemented in scgraph has a time complexity of O((n + m) log n)
    return (m+n)*log(n)

In [36]:
benchmark_data =pamda.read_csv('./outputs/gridscale_time_tests.csv')

for row in benchmark_data:
    n_nodes = int(row['graph_nodes'])
    n_edges = int(row['graph_edges'])
    row['exp_bmssp_to_dijkstra_performance_ratio'] =  bmssp_big_o(n_nodes, n_edges) / dijkstra_big_o(n_nodes, n_edges)
    row['real_bmssp_to_dijkstra_performance_ratio'] = float(row['bmssp_spantree_time_ms']) / float(row['sc_dijkstra_spantree_time_ms'])
    row['pred_bmssp_breakeven_multiplier'] = row['exp_bmssp_to_dijkstra_performance_ratio'] / row['real_bmssp_to_dijkstra_performance_ratio']
    print(row['graph_nodes'], row['pred_bmssp_breakeven_multiplier'])

1089 0.0955058450209039
1089 0.07651320722825761
1089 0.0751533613815157
2500 0.09229976527096381
2500 0.08702917515155519
2500 0.09185415656497802
10000 0.0884630806546585
10000 0.09396390647613466
10000 0.09505834965258342
40000 0.07886211913413384
40000 0.09115290387954267
40000 0.08623704256046401
160000 0.09641688198262151
160000 0.11279192499049989
160000 0.1087808709065895
640000 0.08406903550978478
640000 0.09362595702169524
640000 0.08403568383263839
2560000 0.08519960556609117
2560000 0.09298197743801971
2560000 0.09812982823180112
10240000 0.08316884482024281
10240000 0.09327988993922604
10240000 0.08571925764161932


In [54]:

bmssp_dijkstra_big_o_ratios = []
for magnitude in list(range(1,10)) + list(range(10,300,10)):
    n = 10**magnitude
    connections_per_node = 8
    m = connections_per_node*n
    out ={
        'n_magnitude': f"1e{magnitude}",
        'connections_per_node': connections_per_node,
        'bmssp_to_dijkstra_ratio': bmssp_big_o(n,m)/dijkstra_big_o(n,m)
    }
    bmssp_dijkstra_big_o_ratios.append(out)
    print(out)

pamda.write_csv(filename='./outputs/bmssp_dijkstra_big_o_ratios.csv', data=bmssp_dijkstra_big_o_ratios)


{'n_magnitude': '1e1', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.6694130019924}
{'n_magnitude': '1e2', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.5288639262724771}
{'n_magnitude': '1e3', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.4607580342803859}
{'n_magnitude': '1e4', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.4178243501095243}
{'n_magnitude': '1e5', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.38729718635052657}
{'n_magnitude': '1e6', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.36401788185446704}
{'n_magnitude': '1e7', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.345430602258584}
{'n_magnitude': '1e8', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.3300984976889915}
{'n_magnitude': '1e9', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.31714048804259043}
{'n_magnitude': '1e10', 'connections_per_node': 8, 'bmssp_to_dijkstra_ratio': 0.30598077718536487}
{'n_magnitude': '1e20', 'conn

In [55]:
from math import exp

pred_breakeven_multiplier = sum([row['pred_bmssp_breakeven_multiplier'] for row in benchmark_data]) / len(benchmark_data)
print(f"Predicted breakeven at multiplier = {pred_breakeven_multiplier}")

# TODO: Validate math here.

# Math derivation:
#
# BMSSP(n, m) = m * (ln n)^0.66
# Dijkstra(n, m) = (n + m) * ln n
#
# Assume m = 8n:
#   8n * (ln n)^0.66 = p * (9n * ln n)
#
# Cancel n:
#   8 * (ln n)^0.66 = 9p * ln n
#
# Divide by (ln n)^0.66:
#   8 = 9p * (ln n)^0.34
#
# Solve for ln n:
#   (ln n)^0.34 = 8 / (9p)
#   ln n = (8 / (9p))^(1/0.34)
#
# Finally, to get log10(n):
#   log10(n) = ln(n) / ln(10)

breakeven_magnitude_n = ((8 / (9 * pred_breakeven_multiplier)) ** (1/0.34)) / log(10)

print(f"Predicted breakeven at n = 1e{breakeven_magnitude_n} nodes")


Predicted breakeven at multiplier = 0.09042886128568838
Predicted breakeven at n = 1e360.5959356634255 nodes
